In [2]:
import spacy
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
import os 
import json
from bs4 import BeautifulSoup

nlp = spacy.load("sv_core_news_sm")

def tokenizer(text):
    doc = nlp(text)
    return [token.text for token in doc]

tfidf_vectorizer = TfidfVectorizer(tokenizer=tokenizer)

svd = TruncatedSVD(n_components=128)

log_reg = LogisticRegression(multi_class="multinomial")

<p>Extract the data in a very simple fashion.</p>

In [19]:
speeches = []
parties = []

possible_parties = ["S","M","SD","V","MP","C","KD","L"]

# The directory where data is located
data_dirs = ["speech_data/19_20/","speech_data/20_21/","speech_data/21_22/","speech_data/22_23/"]
year_indices = [0,0,0,0]
speeches_per_year = 5000
data = []
i = 0
for year, data_dir in enumerate(data_dirs):
    year_indices[year] = len(speeches)
    speech_count = 0
    for file in os.listdir(data_dir):
        if file.endswith(".json"):
            if speech_count >= speeches_per_year:
                break
            with open(data_dir+file,"r") as f:
                data = json.load(f)
                speech = data["anforande"]["anforandetext"]
                party = data["anforande"]["parti"]
                # If it is a party who has spoken,
                if party in possible_parties:
                    soup = BeautifulSoup(speech, 'html.parser')
                    speech_text = " ".join([p.get_text() for p in soup.find_all("p")])
                    if speech_text != "":
                        parties.append(party)
                        speeches.append(speech_text)
                        speech_count += 1

print(year_indices)
assert len(parties) == len(speeches)

[0, 5000, 10000, 15000]


In [30]:
print(speeches[:10])
print(parties[:10])

['Fru talman! Vi är i krig mot en osynlig fiende. Vi är i krig mot ett virus - ett virus som snabbt har spridit sig över vår jord och som skördar liv i alla våra världsdelar. Ingen har tidigare mött denna fiende, och ingen är därför heller immun. Inga mediciner finns, och det finns inte heller något vaccin. I går räknade vi till 1\xa0203 döda i Sverige. I världen rapporterar WHO att vi ligger på 125\xa0000 döda, och många fler vet vi att det kommer att bli. Fru talman! Större delen av världen har tvingats till extrema och extraordinära åtgärder. Länder har i princip stängts ned. Folk har förlorat arbetet, många sitter i karantän och alltför många har redan fått sätta livet till. Fallet är och blir hårt. Världen kommer aldrig någonsin att vara densamma igen. Fru talman! Regeringspartierna har arbetat dag och natt de senaste månaderna för att göra det som hör vårt arbete till och minska det antal dödsfall vi kommer att se och det lidande och de ekonomiska förluster som denna pandemi resu

In [17]:
# Turn parties to index
party_to_number = {}
number_to_party = {}
for i, p in enumerate(possible_parties):
    party_to_number[p] = i
    number_to_party[i] = p

print(party_to_number)
print(number_to_party)
print(parties[:20])
parties = [party_to_number.get(p) for p in parties]
print(parties[:20])

{'S': 0, 'M': 1, 'SD': 2, 'V': 3, 'MP': 4, 'C': 5, 'KD': 6, 'L': 7}
{0: 'S', 1: 'M', 2: 'SD', 3: 'V', 4: 'MP', 5: 'C', 6: 'KD', 7: 'L'}
['MP', 'MP', 'M', 'KD', 'M', 'SD', 'SD', 'S', 'MP', 'L', 'S', 'KD', 'L', 'S', 'M', 'S', 'KD', 'SD', 'MP', 'M']
[4, 4, 1, 6, 1, 2, 2, 0, 4, 7, 0, 6, 7, 0, 1, 0, 6, 2, 4, 1]


In [20]:
train_size = speeches_per_year*3
test_size = speeches_per_year

X_train = speeches[:train_size]
y_train = parties[:train_size]

X_test = speeches[train_size:train_size+test_size]
y_test = parties[train_size:train_size+test_size]

In [21]:
X_vectorized = tfidf_vectorizer.fit_transform(X_train)

/home/erko/.local/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [22]:
X_dim_reduced = svd.fit_transform(X_vectorized)

In [24]:
X_train = X_dim_reduced
print(len(X_train))

15000


In [25]:
X_test = svd.transform(tfidf_vectorizer.transform(X_test))

In [29]:
log_reg = LogisticRegression(multi_class="multinomial",max_iter=1000)

In [30]:
log_reg.fit(X_train,y_train)

LogisticRegression(max_iter=1000, multi_class='multinomial')

In [33]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred = log_reg.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           C       0.47      0.09      0.15       393
          KD       0.11      0.01      0.01       601
           L       0.31      0.03      0.05       486
           M       0.16      0.15      0.15      1037
          MP       0.30      0.14      0.19       420
           S       0.21      0.56      0.31      1093
          SD       0.30      0.28      0.29       543
           V       0.46      0.28      0.35       427

    accuracy                           0.23      5000
   macro avg       0.29      0.19      0.19      5000
weighted avg       0.26      0.23      0.19      5000

